In [ ]:
# Input 3D image

axial = np.amax(img, axis=2)
coronal = np.amax(img, axis=1)
sagittal = np.amax(img, axis=0)

axial = np.expand_dims(axial, axis = -1)
axial_C = np.concatenate((axial/np.max(axial), axial/np.max(axial), axial/np.max(axial)), axis = -1)

coronal = np.expand_dims(coronal, axis = -1)
coronal_C = np.concatenate((coronal/np.max(coronal), coronal/np.max(coronal), coronal/np.max(coronal)), axis = -1)

sagittal = np.expand_dims(sagittal, axis = -1)
sagittal_C = np.concatenate((sagittal/np.max(sagittal), sagittal/np.max(sagittal), sagittal/np.max(sagittal)), axis = -1)

axial_C = np.uint8(axial_C * 200)
coronal_C = np.uint8(coronal_C * 200)
sagittal_C = np.uint8(sagittal_C * 200)

output = model.outputs

# Extract layer
last_conv_layer = model.get_layer('c3_1_de')

grads = K.gradients(output, last_conv_layer.output)[0]

pooled_grads = K.mean(grads, axis=(0, 1, 2, 3))

iterate = K.function([*model.inputs], [pooled_grads, last_conv_layer.output[0]])

pooled_grads_value, conv_layer_output_value = iterate([data_input])

# Heatmap
heatmap = np.mean(conv_layer_output_value, axis=-1)

axial_heatmap = np.amax(heatmap, axis=2)
axial_heatmap = np.maximum(axial_heatmap, 0)
axial_heatmap /= np.max(axial_heatmap)

coronal_heatmap = np.amax(heatmap, axis=1)
coronal_heatmap = np.maximum(coronal_heatmap, 0)
coronal_heatmap /= np.max(coronal_heatmap)

sagittal_heatmap = np.amax(heatmap, axis=0)
sagittal_heatmap = np.maximum(sagittal_heatmap, 0)
sagittal_heatmap /= np.max(sagittal_heatmap)

# Axial heatmap
axial_heatmap = cv2.resize(axial_heatmap, (axial_C.shape[1], axial_C.shape[0]))
axial_heatmap = np.uint8(255 * axial_heatmap)
axial_heatmap = cv2.applyColorMap(axial_heatmap, cv2.COLORMAP_JET)
axial_superimposed_img = axial_heatmap * 0.9 + axial_C

# Coronal heatmap
coronal_heatmap = cv2.resize(coronal_heatmap, (coronal_C.shape[1], coronal_C.shape[0]))
coronal_heatmap = np.uint8(255 * coronal_heatmap)
coronal_heatmap = cv2.applyColorMap(coronal_heatmap, cv2.COLORMAP_JET)
coronal_superimposed_img = coronal_heatmap * 0.9 + coronal_C

# Sagittal heatmap
sagittal_heatmap = cv2.resize(sagittal_heatmap, (sagittal_C.shape[1], sagittal_C.shape[0]))
sagittal_heatmap = np.uint8(255 * sagittal_heatmap)
sagittal_heatmap = cv2.applyColorMap(sagittal_heatmap, cv2.COLORMAP_JET)
sagittal_superimposed_img = sagittal_heatmap * 0.9 + sagittal_C

# Save image
coronal_superimposed_img  = ndimage.rotate(coronal_superimposed_img, 90)
fig_name = './Result_grad1/Coronal_' +  str(ep) + '_' + str(ID1) + '.png'
cv2.imwrite(fig_name, coronal_superimposed_img)


sagittal_superimposed_img  = ndimage.rotate(sagittal_superimposed_img, 90)
fig_name = './Result_grad1/Sagittal_' +  str(ep) + '_' + str(ID1) + '.png'
cv2.imwrite(fig_name, sagittal_superimposed_img)

axial_superimposed_img  = ndimage.rotate(axial_superimposed_img, 90)
fig_name = './Result_grad1/Axial_' +  str(ep) + '_' + str(ID1) + '.png'
cv2.imwrite(fig_name, axial_superimposed_img)
